In [ ]:
#Goal: In this project we are trying to figure out which
#Audiobook customer is most likely to buy again using Logistic Regression. 
# The data was gathered over 2 years. The targets was data gathered over
#an additional 6 months, seeing who bought or didn't
#buy in that time. 

In [ ]:
#Import The Relevant Libraries
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import StandardScaler

In [ ]:
#Loading the Raw Data
raw_csv_data= pd.read_csv('Audiobooks-data_raw.csv')
print(display(raw_csv_data.head(20)))

In [ ]:
df=raw_csv_data.copy()
print(display(df.head(20)))

In [ ]:
df=df.drop(['ID'], axis=1)
print(display(df.head()))

In [ ]:
print(df.info())

In [ ]:
#Separate the Targets from the dataset
inputs_all= df.loc[:,'Book length (mins)_overall':'Last visited minus Purchase date']
targets_all= df['Targets']
print(display(inputs_all.head()))
print(display(targets_all.head()))

In [ ]:
#Shuffling the Data to prep for balancing
shuffled_indices= np.arange(inputs_all.shape[0])
np.random.shuffle(shuffled_indices)
shuffled_inputs= inputs_all.iloc[shuffled_indices]
shuffled_targets= targets_all[shuffled_indices]

In [ ]:
#Balance the Dataset
#There are significantly more 0's than 1's in our target.
#We want a good accurate model
print(inputs_all.shape)
print(targets_all.shape)
#%%
num_one_targets= int(np.sum(targets_all))
zero_targets_counter= 0
indices_to_remove= []
print(num_one_targets)
#%%
for i in range(targets_all.shape[0]):
    if targets_all[i]==0:
        zero_targets_counter +=1
        if zero_targets_counter> num_one_targets:
            indices_to_remove.append(i)


In [ ]:
inputs_all_balanced= inputs_all.drop(indices_to_remove,axis=0)
targets_all_balanced= targets_all.drop(indices_to_remove,axis=0)
targets_all_balanced= targets_all_balanced.reset_index(drop=True)

In [ ]:
print(inputs_all_balanced.shape)
print(targets_all_balanced.shape)

In [ ]:
#Standardizing the Inputs
#We won't standardize the Reviews Column because they serve as a 
#categorical variable with a binary 0/1 signifier

print(inputs_all_balanced.columns.values)

In [ ]:
unscaled_inputs= inputs_all_balanced.loc[:,['Book length (mins)_overall', 'Book length (mins)_avg', 'Price_overall',
 'Price_avg', 'Review 10/10', 'Minutes listened', 'Completion',
 'Support requests', 'Last visited minus Purchase date']]

print(display(unscaled_inputs.head()))

In [ ]:
audiobook_scaler= StandardScaler()

In [ ]:
audiobook_scaler.fit(unscaled_inputs)

In [ ]:
scaled_inputs= audiobook_scaler.transform(unscaled_inputs)


In [ ]:
print(display(scaled_inputs))


In [ ]:
print(scaled_inputs.shape)


In [ ]:
#Transforming the scaled_inputs array back into a Dataframe
scaled_inputs= pd.DataFrame(scaled_inputs, columns=['Book length (mins)_overall', 'Book length (mins)_avg',
'Price_overall','Price_avg', 'Review 10/10','Minutes listened', 'Completion','Support requests',
'Last visited minus Purchase date'])

In [ ]:
print(display(scaled_inputs.head()))


In [ ]:
print(display(scaled_inputs.tail(25)))


In [ ]:
#Now adding the 'Review' column to the new scaled inputs Dataframe 
print(inputs_all_balanced['Review'].values)

In [ ]:
review= inputs_all_balanced['Review'].values


In [ ]:
print(review)


In [ ]:
scaled_inputs['Review']= review


In [ ]:
print(display(scaled_inputs.head()))


In [ ]:
print(display(scaled_inputs.tail(25)))


In [ ]:
print(scaled_inputs.info())


In [ ]:
print(scaled_inputs.columns.values)


In [ ]:
reorderd_columns= ['Book length (mins)_overall', 'Book length (mins)_avg', 'Price_overall',
 'Price_avg', 'Review','Review 10/10', 'Minutes listened', 'Completion',
 'Support requests', 'Last visited minus Purchase date']

In [ ]:
scaled_inputs= scaled_inputs[reorderd_columns]
print(display(scaled_inputs.head()))

In [ ]:
#Split the data into Train and Test and Shuffle
#Import relevant library
from sklearn.model_selection import train_test_split
train_test_split(scaled_inputs, targets_all_balanced)

In [ ]:
x_train, x_test, y_train, y_test= train_test_split(scaled_inputs, targets_all_balanced, train_size=0.8, random_state=20)

In [ ]:
#Let's see the shape of these arrays
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
#Logistic Regression with SKLearn
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [ ]:
#Training the Model
reg= LogisticRegression()
reg.fit(x_train, y_train)

In [ ]:
#Checking the accuracy of the model
reg.score(x_train, y_train)

In [ ]:
#Manually Checking the Model Accuracy
model_outputs= reg.predict(x_train)
print(model_outputs)

In [ ]:
print(model_outputs== y_train)


In [ ]:
np.sum(model_outputs==y_train)


In [ ]:
print(model_outputs.shape[0])


In [ ]:
print(np.sum((model_outputs==y_train))/model_outputs.shape[0])


In [ ]:
#Finding the Intercept/Coefficients and Creating a Summary Table
print(reg.intercept_)

In [ ]:
print(reg.coef_)


In [ ]:
print(scaled_inputs.columns.values)


In [ ]:
feature_name= scaled_inputs.columns.values


In [ ]:
summary_table= pd.DataFrame(columns=['Feature name'], data=feature_name)


In [ ]:
summary_table['Coefficient']= np.transpose(reg.coef_)
print(display(summary_table))

In [ ]:
summary_table.index= summary_table.index+1
summary_table.loc[0]= ['Intercept', reg.intercept_[0]]
summary_table= summary_table.sort_index()
print(display(summary_table))

In [ ]:
#Interpreting the Coefficients
summary_table['Odds_ratio']= np.exp(summary_table.Coefficient)

In [ ]:
print(display(summary_table))


In [ ]:
print(display(summary_table.sort_values('Odds_ratio', ascending=False)))


In [ ]:
#So from the odds ratio table it shows that the
#Avg Book Length, Avg Price, Whether customers left reviews
#or sought customer support were the Strongest indicators of 
#Them Buying again.

In [ ]:
#Testing the Model
reg.score(x_test, y_test)


In [ ]:
#Getting the Probability of an output
predicted_proba= reg.predict_proba(x_test)
print(predicted_proba)


In [ ]:
print(predicted_proba.shape)


In [ ]:
print(predicted_proba[:,1])
#If the probability is below 0.5 it gets a 0 and above 0.5, it gets a 1